In [1]:
import cv2
import numpy as np

# Edge detector using Gradient

In [ ]:
def grayscaling(image): # color image를 grayscaling 해주는 함수
  
    gray_image = np.zeros((image.shape[0],image.shape[1]),np.uint8) # 이미지의 크기로 2차원 array를 만들고 요소들을 0으로 초기화

    for i in range(0,image.shape[0]): # height 만큼
        for j in range(0,image.shape[1]): # width 만큼
            b = image[i,j,0] # Blue에 해당하는 값
            g = image[i,j,1] # Green에 해당하는 값
            r = image[i,j,2] # Red에 해당하는 값 
            
            gray_scale = (0.0722 * b  + 0.7152 * g + 0.2126 * r) # HDTV에서 grayscale 할 때 사용하는 계수
            
            if gray_scale > 255: # grayscale 값이 255보다 크면 표현할 수 있는 수의 범위를 넘어 가므로 255 로 지정
                gray_scale = 255
                
            gray_image.itemset(i, j, gray_scale) # 위에서 얻은 grayscale 값들을 2차원 image array에 넣어줌
        
    cv2.imshow('image',gray_image)
    cv2.waitKey(0)
    
    return gray_image


# x 방향으로의 gradient 구하는 함수 (즉, x 방향의 kernel과 x 방향의 픽셀값들 convolution)
def conv3d_x(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB

    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화

    #  이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용, 가장자리가 양쪽 끝 1개씩 이므로, 2개씩 추가
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) 
    padding_image[1:-1, 1:-1,1:-1] = image 
   
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(image.shape[2]): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[0][k] * padding_image[h,w+k,c] # Convolution
                
    return convolve_result


# y 방향으로의 gradient 구하는 함수 (즉, y 방향의 kernel과 y 방향의 픽셀값들 convolution)
def conv3d_y(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB
    
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화
    
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) # 3x3 이므로 양쪽 끝에 하나씩 패딩하므로 원래 이미지보다 +2
    padding_image[1:-1, 1:-1,1:-1] = image 
  
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(color): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[k][0] * padding_image[h+k,w,c] # Convolution 
                
    return convolve_result
        

    
if __name__ == "__main__": 

    # original 이미지를 로드하고 보여줌
    image = cv2.imread('image_test.png') 
    cv2.imshow('original_image',image)
    cv2.waitKey(1)

    #gray_img = grayscaling(image)
    
    # Derivative filter, x축은 1x3 , y축은 3x1 로 kernel 생성, 양 옆 2개의 픽셀의 관계(값 차이)로 필터링  
    x_filter = np.array([[-1,0,1]])
    y_filter = np.array([[-1],[0],[1]])
    
    # convolution 연산을 하기 위해 kernel을 뒤집음
    x_filter = x_filter[:,::-1]
    y_filter = y_filter[::-1] 
    
  # image를 각각 x방향 filter, y방향 filter들과 Convolution
    I_x = conv3d_x(image, x_filter)
    I_y = conv3d_y(image, y_filter)
    
  # 그래디언트의 크기(magnitude)를 계산 
    magnitude = np.sqrt( I_x**2  +  I_y**2 )
    
    threshold = 10 # threshold 값 지정
    
  # threshold 를 적용하여 edge를 검출 
    for h in range(magnitude.shape[0]):
        for w in range(magnitude.shape[1]):
            for color in range(magnitude.shape[2]):
                # threshold 보다 작을 경우 값을 0으로 만들어 edge를 죽이고, threshold 보다 큰 경우만 살림
                if magnitude[h,w,color] < threshold: 
                    magnitude[h,w,color] = 0
                    
                    
    magnitude = magnitude.astype(np.float32) # np.float16은 에러가 발생하기 때문에 np.float32로 dtype 변경
    
    # convertScaleAbs는 각각의 값을 절대값화시키고 정수화함. 이 과정을 거쳐야 출력하였을 때 정상적인 이미지가 나옴
    magnitude = cv2.convertScaleAbs(magnitude, alpha=(255.0)) 
    
    # edge detect된 이미지를 저장 (np.uint8로 변환)
    cv2.imwrite('Gradient.png' , magnitude.astype(np.uint8)) 
   
  
   # edge detect 된 이미지를 보여줌
    cv2.imshow('edge_detected_image', magnitude)
    cv2.waitKey(0)

# Derivative of Box filter

In [ ]:
# 2차원 필터들의 convolution (x 방향)
def conv2d_x(filter1, filter2):
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로

    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
 
  
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height , width + 2)) 
    padding_image[:, 1:-1] = filter2 

    #2차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for k in range(filter1.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                convolve_result[h,w] = convolve_result[h,w] + filter1[0][k] * padding_image[h,w+k] # Convolution
                
    return convolve_result

# 2차원 필터들의 convolution (y 방향)
def conv2d_y(filter1, filter2):
    
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로
    
    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = filter2.shape[0]
 
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width))
    padding_image[1:-1, :] = filter2 

    #2차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for k in range(filter1.size):# 필터의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                convolve_result[h,w] = convolve_result[h,w] + filter1[k][0] * padding_image[h+k,w] # Convolution 
            
    return convolve_result

# x 방향으로의 gradient 구하는 함수 (즉, x 방향의 kernel과 x 방향의 픽셀값들 convolution)
def conv3d_x(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB
     
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) 
    padding_image[1:-1, 1:-1,1:-1] = image 
   
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(image.shape[2]): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[0][k] * padding_image[h,w+k,c]
                
    return convolve_result


# y 방향으로의 gradient 구하는 함수 (즉, y 방향의 kernel과 y 방향의 픽셀값들 convolution)
def conv3d_y(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB
    
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) 
    padding_image[1:-1, 1:-1,1:-1] = image 
  
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(color): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[k][0] * padding_image[h+k,w,c]
                    
    return convolve_result
        
if __name__ == "__main__": 

    # original 이미지를 로드하고 보여줌
    image = cv2.imread('image_test.png') 
   # cv2.imshow('original_image',image)
   # cv2.waitKey(1)

    #gray_img = grayscaling(image)
    
    # Box Filter, 모든 weight들이 평균 값으로 모두 똑같다.
    box_filter_x = 1/3 * np.array([[1,1,1]]) # 1x3 box filter
    box_filter_y = 1/3 * np.array([[1],[1],[1]]) # 3x1 box filter
    
    # Derivative, x축은 1x3 , y축은 3x1 로 kernel 생성, 양 옆 2개의 픽셀의 관계(값 차이)로 필터링  
    x_filter = np.array([[-1,0,1]])
    y_filter = np.array([[-1],[0],[1]])
    
    # convolution 연산을 하기 위해 kernel을 뒤집음
    x_filter = x_filter[:,::-1]
    y_filter = y_filter[::-1] 
    
    # convolution property를 활용하여 이미지에 필터를 적용하기 전에 필터들끼리 컨볼루션 연산(box * derivative)
    x_box_filter =  conv2d_x(x_filter, box_filter_x)
    y_box_filter =  conv2d_y(y_filter, box_filter_y)
   
   #image를 각각 x방향 filter, y방향 filter들과 컨볼루션 연산
    I_x = conv3d_x(image, x_box_filter)
    I_y = conv3d_y(image, y_box_filter)
  
  # 그래디언트의 크기(magnitude)를 계산 
    magnitude = np.sqrt( I_x**2  +  I_y**2 )
    
    threshold = 10 # threshold 값 지정
    
  # threshold 를 적용하여 edge를 검출 
    for h in range(magnitude.shape[0]):
        for w in range(magnitude.shape[1]):
            for color in range(magnitude.shape[2]):
                # threshold 보다 작을 경우 값을 0으로 만들어 edge를 죽이고, threshold 보다 큰 경우만 살림
                if magnitude[h,w,color] < threshold: 
                    magnitude[h,w,color] = 0
                          
    magnitude = magnitude.astype(np.float32) # np.float16은 에러가 발생하기 때문에 np.float32로 dtype 변경
    
    # convertScaleAbs는 각각의 값을 절대값화시키고 정수화함. 이 과정을 거쳐야 출력하였을 때 정상적인 이미지가 나옴
    magnitude = cv2.convertScaleAbs(magnitude, alpha=(255.0)) 
    
    # edge detect된 이미지를 저장 (np.uint8로 변환)
    cv2.imwrite('Derivative of Boxfilter.png' , magnitude.astype(np.uint8)) 
   
  
   # edge detect 된 이미지를 보여줌
    cv2.imshow('edge_detected_image', magnitude)
    cv2.waitKey(0)

# DoG (Derivative of Gaussian)

In [2]:
import time

def derivative_x_filter(size):
    
    deriv_x_filter = np.zeros(size)
    h , w = size
    
    # 3x3 array에서 x방향 부분 양쪽을 -1, 1 을 줌 
    deriv_x_filter[h//2][0]= -1 
    deriv_x_filter[h//2][-1] = 1
  
    return deriv_x_filter


def derivative_y_filter(size):
    
    deriv_y_filter = np.zeros(size)
    h , w = size
    
     # 3x3 array에서 y방향 부분 양쪽을 -1, 1 을 줌 
    deriv_y_filter[0][h//2]= -1
    deriv_y_filter[-1][h//2] = 1

    return deriv_y_filter

def gaussian_filter(shape, sigma):
 
    h = (shape[0] - 1) / 2
    w = (shape[1] - 1) / 2

    y, x = np.ogrid[-h:h + 1, -w:w + 1] # np.ogrid[-n:n+] : -n~n까지 증가하는 array를 반환한다.

    gaussian_filter = 1/(2*np.pi*(sigma**2))*np.exp(-1*(x**2+y**2)/(2*(sigma**2)))
    gaussian_filter /= gaussian_filter.sum() # 정규화
    
    return gaussian_filter


 # 2차원 필터들의 convolution을 위한 함수
def conv2d(filter1, filter2):
    
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로

    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = filter2.shape[0]//2

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size)) 
    padding_image[size:-1*size, size:-1*size] = filter2 

    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            convolve_result[h,w] = (filter1 * padding_image[h: h+3, w: w+3]).sum()
            
    return convolve_result


 #3차원 RGB 포함한 convolution
def conv3d(image, kernel):
    
    height = image.shape[0] # 이미지의 세로
    width = image.shape[1] # 이미지의 가로
    color = image.shape[2] # RGB
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = kernel.shape[0]//2
    
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size, color + 2*size)) 
    padding_image[size:-1*size, size:-1*size,size:-1*size] = image 
    
   
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for i in range(3): # RGB 채널의 수만큼
                convolve_result[h,w,i] = (kernel * padding_image[h: h + kernel.shape[0], w: w + kernel.shape[0], i]).sum() # convolution
              
    return convolve_result



def conv2d_x(filter1, filter2):
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로

    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
 
  
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height , width + 2)) 
    padding_image[:, 1:-1] = filter2 

    #2차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for k in range(filter1.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                convolve_result[h,w] = convolve_result[h,w] + filter1[0][k] * padding_image[h,w+k] # Convolution
                
    return convolve_result

# 2차원 필터들의 convolution (y 방향)
def conv2d_y(filter1, filter2):
    
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로
    
    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = filter2.shape[0]
 
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width))
    padding_image[1:-1, :] = filter2 

    #2차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for k in range(filter1.size):# 필터의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                convolve_result[h,w] = convolve_result[h,w] + filter1[k][0] * padding_image[h+k,w] # Convolution 
            
    return convolve_result

# x 방향으로의 gradient 구하는 함수 (즉, x 방향의 kernel과 x 방향의 픽셀값들 convolution)
def conv3d_x(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB
     
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) 
    padding_image[1:-1, 1:-1,1:-1] = image 
   
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(image.shape[2]): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[0][k] * padding_image[h,w+k,c] # convolution
                
    return convolve_result


# y 방향으로의 gradient 구하는 함수 (즉, y 방향의 kernel과 y 방향의 픽셀값들 convolution)
def conv3d_y(image, kernel):
    
    height = image.shape[0] # 이미지의 세로 (y 방향)
    width = image.shape[1] # 이미지의 가로 (x 방향)
    color = image.shape[2] # RGB
    
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기와 형태로 array 생성하면서 0으로 초기화

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2 , width + 2, 3 + 2)) # 3x3 이므로 패딩은 양쪽에 kernel size//2 만큼 적용
    padding_image[1:-1, 1:-1,1:-1] = image 
  
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for c in range(color): # RGB 채널의 수만큼
                for k in range(kernel.size):# 커널의 사이즈만큼 (Convolution 연산의 결과 값들을 다 더해주기 위함)
                    convolve_result[h,w,c] = convolve_result[h,w,c] + kernel[k][0] * padding_image[h+k,w,c] # convolution
                    
    return convolve_result

  
if __name__ == "__main__": 

    # original 이미지를 로드하고 보여줌
    image = cv2.imread('image_test.png') 
   # cv2.imshow('original_image',image)
   # cv2.waitKey(1)
    
    sigma = 2 # sigma 값 지정
    
    # gaussian filter , 가우시안 분포 모양으로 weight를 부여
    # 1D, 2D 가우시안 필터 모두 사용하여 성능 비교
    
    gaussian_filter_x_2d =  gaussian_filter((3,3),sigma)
    gaussian_filter_y_2d =  gaussian_filter((3,3),sigma)
    gaussian_filter_x_1d =  gaussian_filter((1,3),sigma)
    gaussian_filter_y_1d =  gaussian_filter((3,1),sigma)

    # derivative filter
    deriv_x_filter_2d = derivative_x_filter((3,3)) # 3x3 x derivative filter 
    deriv_y_filter_2d = derivative_y_filter((3,3))
    deriv_x_filter_1d = np.array([[-1,0,1]])
    deriv_y_filter_1d = np.array([[-1],[0],[1]])
    
    start = time.perf_counter() # start time
    
    x_filter_1d = conv2d_x(deriv_x_filter_1d , gaussian_filter_x_1d)
    y_filter_1d = conv2d_y(deriv_y_filter_1d , gaussian_filter_y_1d)
    x_filter_1d = x_filter_1d[:,::-1]
    y_filter_1d = y_filter_1d[::-1] 
    
    I_x_1d = conv3d_x(image,x_filter_1d)
    I_y_1d = conv3d_y(image,y_filter_1d)
    
    end = time.perf_counter() #end time
    print("1D:", end - start)
    
    start = time.perf_counter()
    
    # convolution property를 활용하여 이미지에 필터를 적용하기 전에 필터들끼리 컨볼루션 연산(derivative * gaussian)   
    x_filter_2d = conv2d(deriv_x_filter_2d , gaussian_filter_x_2d)
    y_filter_2d = conv2d(deriv_y_filter_2d , gaussian_filter_y_2d)
    #x_filter_2d = deriv_x_filter_2d * gaussian_filter_x_2d
    #y_filter_2d = deriv_y_filter_2d * gaussian_filter_y_2d
    
    # convolution 연산을 하기 위해 kernel을 뒤집음
    x_filter_2d = x_filter_2d[:,::-1]
    y_filter_2d = y_filter_2d[::-1] 
   
  # image를 각각 x방향 filter, y방향 filter들과 컨볼루션 연산
    I_x_2d = conv3d(image, x_filter_2d) # 2d 로 convolution 한 것을 input으로 넣음
    I_y_2d = conv3d(image, y_filter_2d)
    
    end = time.perf_counter()
    print("2D:", end - start)
    

    
  # 그래디언트의 크기(magnitude)를 계산 
    magnitude = np.sqrt( I_x_2d**2  +  I_y_2d**2 ) # 2d convolution (3x3) 을 한 것이 성능이 더 좋으므로 2d convolution의 결과를 사용
    
    threshold = 8 # threshold 값 지정

  # threshold 를 적용하여 edge를 검출 
    for h in range(magnitude.shape[0]): # 이미지의 세로
        for w in range(magnitude.shape[1]): #이미지의 가로
            for color in range(magnitude.shape[2]): #이미지의 RGB채널 수 
                # threshold 보다 작을 경우 값을 0으로 만들어 edge를 죽이고, threshold 보다 큰 경우만 살림
                if magnitude[h,w,color] < threshold: 
                    magnitude[h,w,color] = 0

                          
    magnitude = magnitude.astype(np.float32) # np.float16은 에러가 발생하기 때문에 np.float32로 dtype 변경
    
    # convertScaleAbs는 각각의 값을 절대값화시키고 정수화함. 이 과정을 거쳐야 출력하였을 때 정상적인 이미지가 나옴
    magnitude = cv2.convertScaleAbs(magnitude, alpha=(255.0)) 
    
    # edge detect된 이미지를 저장 (np.uint8로 변환)
    cv2.imwrite('Derivative of Gaussian.png' , magnitude.astype(np.uint8)) 
   
  
   # edge detect 된 이미지를 보여줌
    cv2.imshow('edge_detected_image', magnitude)
    cv2.waitKey(0)

1D: 9.1385127
2D: 4.3678691999999995


# Laplacian of Gaussian (LoG)

In [3]:
# 라플라시안 x방향 필터 생성 (3x3)
def laplacian_x_filter(size):
    
    lap_x_filter = np.zeros(size)
    h , w = size
    
    # x축 방향으로 1,-2,1
    lap_x_filter[h//2][0]= 1
    lap_x_filter[h//2][1] = -2
    lap_x_filter[h//2][-1] = 1
  
    return lap_x_filter

# 라플라시안 y방향 필터 생성 (3x3)
def laplacian_y_filter(size):
    
    lap_y_filter = np.zeros(size)
    h , w = size
    
    # y축 방향으로 1,-2,1 
    lap_y_filter[0][h//2]= 1
    lap_y_filter[1][h//2] = -2
    lap_y_filter[-1][h//2] = 1

    return lap_y_filter

# 2D 가우시안 필터 생성 (3x3)
def gaussian_filter(kernel_size,sigma):
    
    size = kernel_size//2
    y, x = np.ogrid[-size:size+1, -size:size+1] # np.ogrid[-n:n+] : -n~n까지 증가하는 array를 반환한다.
    
    gaussian_filter = 1/(2*np.pi*(sigma**2))*np.exp(-1*(x**2+y**2)/(2*(sigma**2))) # 가우시안 공식 적용
    gaussian_filter /= gaussian_filter.sum() # 정규화
    
    return gaussian_filter
    
    

 # 2차원 필터들의 convolution을 위한 함수
def conv2d(filter1, filter2):
    
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로

    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = filter2.shape[0]//2
 
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size)) 
    padding_image[size:-1*size, size:-1*size] = filter2 

    # 2차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            convolve_result[h,w] = (filter1 * padding_image[h: h+3, w: w+3]).sum() # Convolution
            
    return convolve_result    
    

 #3차원 RGB 포함한 convolution
def conv3d(image, kernel):
    
    height = image.shape[0] # 이미지의 세로
    width = image.shape[1] # 이미지의 가로
    color = image.shape[2] # RGB
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
    
    size = kernel.shape[0]//2 

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size, color + 2*size)) 
    padding_image[size:-1*size, size:-1*size,size:-1*size] = image 
    
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for i in range(3): # RGB 채널의 수만큼
                convolve_result[h,w,i] = (kernel * padding_image[h: h + kernel.shape[0], w: w + kernel.shape[0], i]).sum() # convolution
                
    return convolve_result



if __name__ == "__main__": 

    # original 이미지를 로드하고 보여줌
    image = cv2.imread('image_test.png') 
    #cv2.imshow('original_image',image)
    #cv2.waitKey(1)

    
    sigma = 1 # sigma 값 지정
        
    # gaussian filter , 가우시안 정규 분포 모양으로 weight를 부여
    gaussian_filter_x =  gaussian_filter(3,sigma)
    gaussian_filter_y =  gaussian_filter(3,sigma)
    
    # laplacian filter
    lap_x_filter = laplacian_x_filter((3,3)) # 3x3 x laplacian filter
    lap_y_filter = laplacian_y_filter((3,3)) # 3x3 x laplacian filter
    
    laplacian_filter = lap_x_filter + lap_y_filter
    
    # convolution property를 활용하여 이미지에 필터를 적용하기 전에 필터들끼리 컨볼루션 연산(laplacian * gaussian)   
    LoG_filter = conv2d(gaussian_filter_x, laplacian_filter)
    
   
  # image를 각각 x방향 filter, y방향 filter들과 컨볼루션 연산
    I_x = conv3d(image, LoG_filter)
    I_y = conv3d(image, LoG_filter)
 
  # second derivative 를 통해 얻은 scalar 
    magnitude = I_x + I_y
    
  # threshold 를 적용하여 edge를 검출 
    for h in range(magnitude.shape[0]-1): # 이미지의 세로
        for w in range(magnitude.shape[1]-1): #이미지의 가로
            for color in range(magnitude.shape[2]): #이미지의 RGB채널 수 
                # threshold 보다 작을 경우 값을 0으로 만들어 edge를 죽이고, threshold 보다 큰 경우만 살림
                if magnitude[h,w,color] > 128:
                    if magnitude[h+1,w,color] < 128 or magnitude[h,w+1,color] < 128 or magnitude[h+1,w+1,color] < 128: 
                        magnitude[h,w,color] = 128
                elif magnitude[h,w,color] < 128:
                    if magnitude[h+1,w,color] > 128 or magnitude[h,w+1,color] > 128 or magnitude[h+1,w+1,color] > 128:
                        magnitude[h,w,color] = 128
                
    
    for h in range(magnitude.shape[0]): # 이미지의 세로
        for w in range(magnitude.shape[1]): #이미지의 가로
            for color in range(magnitude.shape[2]):
                if magnitude[h,w,color] != 128:
                    magnitude[h,w,color] = 0
                    
                    
    magnitude = magnitude.astype(np.float32) # np.float16은 에러가 발생하기 때문에 np.float32로 dtype 변경
    
    # convertScaleAbs는 각각의 값을 절대값화시키고 정수화함. 이 과정을 거쳐야 출력하였을 때 정상적인 이미지가 나옴
    magnitude = cv2.convertScaleAbs(magnitude, alpha=(255.0)) 
    
    # edge detect된 이미지를 저장 (np.uint8로 변환)
    cv2.imwrite('Laplacian of Gaussian.png' , magnitude.astype(np.uint8)) 
   
  
   # edge detect 된 이미지를 보여줌
    cv2.imshow('edge_detected_image', magnitude)
    cv2.waitKey(0)

# Difference of Gaussian(DoG)

In [8]:
def gaussian_filter(shape, sigma):
 
    h = (shape[0] - 1) / 2
    w = (shape[1] - 1) / 2

    y, x = np.ogrid[-h:h + 1, -w:w + 1] # np.ogrid[-n:n+] : -n~n까지 증가하는 array를 반환한다.

    gaussian_filter = 1/(2*np.pi*(sigma**2))*np.exp(-1*(x**2+y**2)/(2*(sigma**2)))
    gaussian_filter /= gaussian_filter.sum() # 정규화
    
    return gaussian_filter


 # 2차원 필터들의 convolution을 위한 함수
def conv2d(filter1, filter2):
    
    height = filter2.shape[0] # 이미지의 세로
    width = filter2.shape[1] # 이미지의 가로

    convolve_result = np.zeros_like(filter2) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = filter2.shape[0]//2

    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size)) 
    padding_image[size:-1*size, size:-1*size] = filter2 

    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            convolve_result[h,w] = (filter1 * padding_image[h: h+3, w: w+3]).sum()
            
    return convolve_result


 #3차원 RGB 포함한 convolution
def conv3d(image, kernel):
    
    height = image.shape[0] # 이미지의 세로
    width = image.shape[1] # 이미지의 가로
    color = image.shape[2] # RGB
    convolve_result = np.zeros_like(image) # 이미지와 동일한 크기로 array 생성하면서 0으로 초기화
   
    size = kernel.shape[0]//2
    
    # # 이미지의 출력 크기가 입력과 동일하게 하기 위해 입력 이미지에 zero-padding 적용
    padding_image = np.zeros((height + 2*size , width + 2 * size, color + 2*size)) 
    padding_image[size:-1*size, size:-1*size,size:-1*size] = image 
    
   
    #3차원 array의 매 픽셀마다 kernel과 input image의 Convolution 연산의 결과값을 넣어줌
    for h in range(height):
        for w in range(width):
            for i in range(3): # RGB 채널의 수만큼
                convolve_result[h,w,i] = (kernel * padding_image[h: h + kernel.shape[0], w: w + kernel.shape[0], i]).sum() # convolution
              
    return convolve_result


if __name__ == "__main__": 

    # original 이미지를 로드하고 보여줌
    image = cv2.imread('image_test.png') 
   # cv2.imshow('original_image',image)
   # cv2.waitKey(1)
    
    # sigma 값 지정
    sigma1 = 1
    sigma2 = 2

     # 2개의 gaussian filter의 차이로 필터를 만듦
    diff = gaussian_filter((3,3),sigma1) - gaussian_filter((3,3),sigma2)
    
    # convolution 연산을 하기 위해 kernel을 뒤집음
    diff_x = diff[:,::-1]
    diff_y = diff[::-1] 
  
  # image를 각각 x방향 filter, y방향 filter들과 컨볼루션 연산
    I_x = conv3d(image, diff_x)
    I_y = conv3d(image, diff_y)
   

  # second derivative 를 통해 얻은 scalar 
    magnitude = I_x + I_y
    
  # threshold 를 적용하여 edge를 검출 
    for h in range(magnitude.shape[0]-1): # 이미지의 세로
        for w in range(magnitude.shape[1]-1): #이미지의 가로
            for color in range(magnitude.shape[2]): #이미지의 RGB채널 수 
                # threshold 보다 작을 경우 값을 0으로 만들어 edge를 죽이고, threshold 보다 큰 경우만 살림
                if magnitude[h,w,color] > 128:
                    if magnitude[h+1,w,color] < 128 or magnitude[h,w+1,color] < 128 or magnitude[h+1,w+1,color] < 128: 
                        magnitude[h,w,color] = 128
                elif magnitude[h,w,color] < 128:
                    if magnitude[h+1,w,color] > 128 or magnitude[h,w+1,color] > 128 or magnitude[h+1,w+1,color] > 128:
                        magnitude[h,w,color] = 128
                
    
    for h in range(magnitude.shape[0]): # 이미지의 세로
        for w in range(magnitude.shape[1]): #이미지의 가로
            for color in range(magnitude.shape[2]):
                if magnitude[h,w,color] != 128:
                    magnitude[h,w,color] = 0

                    
                   
    magnitude = magnitude.astype(np.float32) # np.float16은 에러가 발생하기 때문에 np.float32로 dtype 변경
    
    # convertScaleAbs는 각각의 값을 절대값화시키고 정수화함. 이 과정을 거쳐야 출력하였을 때 정상적인 이미지가 나옴
    magnitude = cv2.convertScaleAbs(magnitude, alpha=(255.0)) 
    
    # edge detect된 이미지를 저장 (np.uint8로 변환)
    cv2.imwrite('Difference of Gaussian.png' , magnitude.astype(np.uint8)) 
   
  
   # edge detect 된 이미지를 보여줌
    cv2.imshow('edge_detected_image', magnitude)
    cv2.waitKey(0)